<a href="https://colab.research.google.com/github/jaygo-kr/brunch/blob/master/01_01_get_naver_best_music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 뮤직 - 시대별 음악 크롤링하기
[시대별 음악, 추천음악 : 네이버 뮤직](https://music.naver.com/recommend/timeLine.nhn)

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import time
from google.colab import files, drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
ERA_DICT = {1980:'1980',1990:'1990',2000:'2000',2010:'2010'}
GENRE_DICT = {'K01':'발라드','K02':'댄스','K03':'힙합','K04':'록/포크','K05':'트로트'}
COLUMNS_LIST = ['곡명','아티스트','앨범','시대','장르','순위/전체','순위/시대','순위/장르','순위/시대/장르']

## 1. 시대, 장르별로 구분하여 순위대로 추출

In [0]:
def read_data_seperate(ERA, GENRE, DISPLAY = 50):
  initial_url = 'https://music.naver.com/recommend/trackTimeLine.nhn?fromTime={ERA_from}&toTime={ERA_to}&searchType=track&orderType=score&viewType=thumbnail&genre={GENRE}&display={DISPLAY}'.format(
    ERA_from=ERA, ERA_to=ERA+10, GENRE=GENRE, DISPLAY=DISPLAY)
  r = requests.get(initial_url)
  
  html = r.text
  soup = BeautifulSoup(html, 'html.parser')
  
  num_select = soup.select(
    'div.sorting_area > div.tab_rdo > label.item10 > em'
  )
  num = int(num_select[0].text[1:-1])
  DISPLAY = num
  
  target_url = 'https://music.naver.com/recommend/trackTimeLine.nhn?fromTime={ERA_from}&toTime={ERA_to}&searchType=track&orderType=score&viewType=thumbnail&genre={GENRE}&display={DISPLAY}'.format(
    ERA_from=ERA, ERA_to=ERA+10, GENRE=GENRE, DISPLAY=DISPLAY)
  r = requests.get(target_url)
  
  print('STEP {},{} : {} data reading...'.format(ERA, GENRE, num))
  
  html = r.text
  soup = BeautifulSoup(html, 'html.parser')
  

  title_select = soup.select(
    'tbody > tr > td.name'
  )
  artist_select = soup.select(
    'tbody > tr > td._artist'
  )
  album_select = soup.select(
    'tbody > tr > td.album > a._album'
  )

  data = pd.DataFrame(columns = COLUMNS_LIST[:5] + [COLUMNS_LIST[8]])

  #for col in COLUMNS_LIST:
  title_list = []
  for select in title_select[1:]:
    title_list.append(select.text.strip())
  data[COLUMNS_LIST[0]] = title_list
  
  artist_list = []
  for select in artist_select[1:]:
    artist_list.append(select.text.strip())
  data[COLUMNS_LIST[1]] = artist_list
  
  album_list = []
  for select in album_select:
    album_list.append(select.text)
  data[COLUMNS_LIST[2]] = album_list
  
  era_list = [ERA_DICT[ERA] for _ in range(num)]
  data[COLUMNS_LIST[3]] = era_list
  
  genre_list = [GENRE_DICT[GENRE] for _ in range(num)]
  data[COLUMNS_LIST[4]] = genre_list
  
  rank_list = [i+1 for i in range(num)]
  data[COLUMNS_LIST[8]] = pd.to_numeric(rank_list)
  print('STEP {},{} : {} data done.'.format(ERA, GENRE, num))
  return data 

In [4]:
full_data_sep = pd.DataFrame()

for era in ERA_DICT.keys():
  for genre in GENRE_DICT.keys():
    read_data = read_data_seperate(era, genre)
    full_data_sep = pd.concat([full_data_sep, read_data])

print("Total data : {}".format(len(full_data_sep)))

STEP 1980,K01 : 1005 data reading...
STEP 1980,K01 : 1005 data done.
STEP 1980,K02 : 97 data reading...
STEP 1980,K02 : 97 data done.
STEP 1980,K03 : 0 data reading...
STEP 1980,K03 : 0 data done.
STEP 1980,K04 : 1002 data reading...
STEP 1980,K04 : 1002 data done.
STEP 1980,K05 : 1001 data reading...
STEP 1980,K05 : 1001 data done.
STEP 1990,K01 : 1000 data reading...
STEP 1990,K01 : 1000 data done.
STEP 1990,K02 : 1000 data reading...
STEP 1990,K02 : 1000 data done.
STEP 1990,K03 : 519 data reading...
STEP 1990,K03 : 519 data done.
STEP 1990,K04 : 1000 data reading...
STEP 1990,K04 : 1000 data done.
STEP 1990,K05 : 1000 data reading...
STEP 1990,K05 : 1000 data done.
STEP 2000,K01 : 1000 data reading...
STEP 2000,K01 : 1000 data done.
STEP 2000,K02 : 1000 data reading...
STEP 2000,K02 : 1000 data done.
STEP 2000,K03 : 1000 data reading...
STEP 2000,K03 : 1000 data done.
STEP 2000,K04 : 1000 data reading...
STEP 2000,K04 : 1000 data done.
STEP 2000,K05 : 1000 data reading...
STEP 2000

In [5]:
full_data_sep.head()

,곡명,아티스트,앨범,시대,장르,순위/시대/장르
0,가로수 그늘아래 서면,이문세,5집 가로수 그늘아래서면,1980,발라드,1
1,사랑하기 때문에,유재하,1집 사랑하기 때문에,1980,발라드,2
2,너에게로 또 다시,변진섭,2집 너에게로 또다시,1980,발라드,3
3,가리워진 길,유재하,1집 사랑하기 때문에,1980,발라드,4
4,비처럼 음악처럼,김현식,3집 비처럼 음악처럼,1980,발라드,5


In [0]:
full_data_sep.to_csv("./gdrive/My Drive/Colab/data/full_data_sep.csv", index=False)

## 2. 시대별로 구분하여 순위대로 추출

In [0]:
def read_data_era(ERA, DISPLAY = 50):
  initial_url = 'https://music.naver.com/recommend/trackTimeLine.nhn?fromTime={ERA_from}&toTime={ERA_to}&searchType=track&orderType=score&viewType=thumbnail&genre={GENRE}&display={DISPLAY}'.format(
    ERA_from=ERA, ERA_to=ERA+10, GENRE=','.join(GENRE_DICT.keys()), DISPLAY=DISPLAY)
  r = requests.get(initial_url)
  
  html = r.text
  soup = BeautifulSoup(html, 'html.parser')
  
  num_select = soup.select(
    'div.sorting_area > div.tab_rdo > label.item10 > em'
  )
  num = int(num_select[0].text[1:-1])
  DISPLAY = num
  
  target_url = 'https://music.naver.com/recommend/trackTimeLine.nhn?fromTime={ERA_from}&toTime={ERA_to}&searchType=track&orderType=score&viewType=thumbnail&genre={GENRE}&display={DISPLAY}'.format(
    ERA_from=ERA, ERA_to=ERA+10, GENRE=','.join(GENRE_DICT.keys()), DISPLAY=num)
  r = requests.get(target_url)
  print('STEP {} : {} data reading...'.format(ERA, num))
  
  html = r.text
  soup = BeautifulSoup(html, 'html.parser')
  
  title_select = soup.select(
    'tbody > tr > td.name'
  )
  artist_select = soup.select(
    'tbody > tr > td._artist'
  )
  album_select = soup.select(
    'tbody > tr > td.album > a._album'
  )
  
  data = pd.DataFrame(columns = COLUMNS_LIST[:4] + [COLUMNS_LIST[6]])

  #for col in COLUMNS_LIST:
  title_list = []
  for select in title_select[1:]:
    title_list.append(select.text.strip())
  data[COLUMNS_LIST[0]] = title_list
  
  artist_list = []
  for select in artist_select[1:]:
    artist_list.append(select.text.strip())
  data[COLUMNS_LIST[1]] = artist_list
  
  album_list = []
  for select in album_select:
    album_list.append(select.text)
  data[COLUMNS_LIST[2]] = album_list
  
  era_list = [ERA_DICT[ERA] for _ in range(num)]
  data[COLUMNS_LIST[3]] = era_list
  
  rank_list = [i+1 for i in range(num)]
  data[COLUMNS_LIST[6]] = pd.to_numeric(rank_list)
  print('STEP {} : {} data done.'.format(ERA, num))
  
  return data  

In [8]:
full_data_era = pd.DataFrame()

for era in ERA_DICT.keys():
  read_data = read_data_era(era)
  full_data_era = pd.concat([full_data_era, read_data])

print("Total data : {}".format(len(full_data_era)))

STEP 1980 : 3105 data reading...
STEP 1980 : 3105 data done.
STEP 1990 : 4519 data reading...
STEP 1990 : 4519 data done.
STEP 2000 : 5000 data reading...
STEP 2000 : 5000 data done.
STEP 2010 : 5000 data reading...
STEP 2010 : 5000 data done.
Total data : 17624


In [9]:
full_data_era.head()

,곡명,아티스트,앨범,시대,순위/시대
0,가로수 그늘아래 서면,이문세,5집 가로수 그늘아래서면,1980,1
1,사랑하기 때문에,유재하,1집 사랑하기 때문에,1980,2
2,너에게로 또 다시,변진섭,2집 너에게로 또다시,1980,3
3,가리워진 길,유재하,1집 사랑하기 때문에,1980,4
4,비처럼 음악처럼,김현식,3집 비처럼 음악처럼,1980,5


In [0]:
full_data_era.to_csv("./gdrive/My Drive/Colab/data/full_data_era.csv", index=False)

## 3. 장르별로 구분하여 순위대로 추출

In [0]:
def read_data_genre(GENRE, DISPLAY = 50):
  initial_url = 'https://music.naver.com/recommend/trackTimeLine.nhn?fromTime={ERA_from}&toTime={ERA_to}&searchType=track&orderType=score&viewType=thumbnail&genre={GENRE}&display={DISPLAY}'.format(
    ERA_from=1980, ERA_to=2010+10, GENRE=GENRE, DISPLAY=DISPLAY)
  r = requests.get(initial_url)
  
  html = r.text
  soup = BeautifulSoup(html, 'html.parser')
  
  num_select = soup.select(
    'div.sorting_area > div.tab_rdo > label.item10 > em'
  )
  num = int(num_select[0].text[1:-1])
  DISPLAY = num
  
  target_url = 'https://music.naver.com/recommend/trackTimeLine.nhn?fromTime={ERA_from}&toTime={ERA_to}&searchType=track&orderType=score&viewType=thumbnail&genre={GENRE}&display={DISPLAY}'.format(
    ERA_from=1980, ERA_to=2010+10, GENRE=GENRE, DISPLAY=num)
  r = requests.get(target_url)
  print('STEP {} : {} data reading...'.format(GENRE, num))
  
  html = r.text
  soup = BeautifulSoup(html, 'html.parser')
  
  title_select = soup.select(
    'tbody > tr > td.name'
  )
  artist_select = soup.select(
    'tbody > tr > td._artist'
  )
  album_select = soup.select(
    'tbody > tr > td.album > a._album'
  )
  
  data = pd.DataFrame(columns = COLUMNS_LIST[:3] + [COLUMNS_LIST[4]] + [COLUMNS_LIST[7]])

  #for col in COLUMNS_LIST:
  title_list = []
  for select in title_select[1:]:
    title_list.append(select.text.strip())
  data[COLUMNS_LIST[0]] = title_list
  
  artist_list = []
  for select in artist_select[1:]:
    artist_list.append(select.text.strip())
  data[COLUMNS_LIST[1]] = artist_list
  
  album_list = []
  for select in album_select:
    album_list.append(select.text)
  data[COLUMNS_LIST[2]] = album_list
  
  genre_list = [GENRE_DICT[GENRE] for _ in range(num)]
  data[COLUMNS_LIST[4]] = genre_list
  
  rank_list = [i+1 for i in range(num)]
  data[COLUMNS_LIST[7]] = pd.to_numeric(rank_list)
  print('STEP {} : {} data done.'.format(GENRE, num))
  
  return data  

In [12]:
full_data_genre = pd.DataFrame()

for genre in GENRE_DICT.keys():
  read_data = read_data_genre(genre)
  full_data_genre = pd.concat([full_data_genre, read_data])

print("Total data : {}".format(len(full_data_genre)))

STEP K01 : 4005 data reading...
STEP K01 : 4005 data done.
STEP K02 : 3097 data reading...
STEP K02 : 3097 data done.
STEP K03 : 2519 data reading...
STEP K03 : 2519 data done.
STEP K04 : 4002 data reading...
STEP K04 : 4002 data done.
STEP K05 : 4001 data reading...
STEP K05 : 4001 data done.
Total data : 17624


In [13]:
full_data_genre.head()

,곡명,아티스트,앨범,장르,순위/장르
0,소주 한 잔,임창정,10집 Bye,발라드,1
1,그대네요 (With 아이유),성시경,그대네요,발라드,2
2,눈의 꽃,박효신,미안하다 사랑한다 OST (KBS 미니시리즈),발라드,3
3,벌써 일년,브라운 아이즈,1집 Brown Eyes,발라드,4
4,어떻게 사랑이 그래요,이승환,9집 Hwantastic,발라드,5


In [0]:
full_data_genre.to_csv("./gdrive/My Drive/Colab/data/full_data_genre.csv", index=False)

## 4. 전체 리스트 순위대로 추출

In [0]:
def read_data_all(DISPLAY = 50):
  initial_url = 'https://music.naver.com/recommend/trackTimeLine.nhn?fromTime={ERA_from}&toTime={ERA_to}&searchType=track&orderType=score&viewType=thumbnail&genre={GENRE}&display={DISPLAY}'.format(
    ERA_from=1980, ERA_to=2010+10, GENRE=','.join(GENRE_DICT.keys()), DISPLAY=DISPLAY)
  r = requests.get(initial_url)
  
  html = r.text
  soup = BeautifulSoup(html, 'html.parser')
  
  num_select = soup.select(
    'div.sorting_area > div.tab_rdo > label.item10 > em'
  )
  num = int(num_select[0].text[1:-1])
  DISPLAY = num
  
  target_url = 'https://music.naver.com/recommend/trackTimeLine.nhn?fromTime={ERA_from}&toTime={ERA_to}&searchType=track&orderType=score&viewType=thumbnail&genre={GENRE}&display={DISPLAY}'.format(
    ERA_from=1980, ERA_to=2010+10, GENRE=','.join(GENRE_DICT.keys()), DISPLAY=num)
  r = requests.get(target_url)
  print('STEP ALL : {} data reading...'.format(num))
  
  html = r.text
  soup = BeautifulSoup(html, 'html.parser')
  
  title_select = soup.select(
    'tbody > tr > td.name'
  )
  artist_select = soup.select(
    'tbody > tr > td._artist'
  )
  album_select = soup.select(
    'tbody > tr > td.album > a._album'
  )
  
  data = pd.DataFrame(columns = COLUMNS_LIST[:3] + [COLUMNS_LIST[5]])

  #for col in COLUMNS_LIST:
  title_list = []
  for select in title_select[1:]:
    title_list.append(select.text.strip())
  data[COLUMNS_LIST[0]] = title_list
  
  artist_list = []
  for select in artist_select[1:]:
    artist_list.append(select.text.strip())
  data[COLUMNS_LIST[1]] = artist_list
  
  album_list = []
  for select in album_select:
    album_list.append(select.text)
  data[COLUMNS_LIST[2]] = album_list
  
  rank_list = [i+1 for i in range(num)]
  data[COLUMNS_LIST[5]] = pd.to_numeric(rank_list)
  print('STEP ALL : {} data done.'.format(num))
  
  return data  

In [16]:
full_data_all = pd.DataFrame()

full_data_all = read_data_all()

print("Total data : {}".format(len(full_data_all)))

STEP ALL : 17624 data reading...
STEP ALL : 17624 data done.
Total data : 17624


In [17]:
full_data_all.tail()

,곡명,아티스트,앨범,순위/전체
17619,J2y,Z.A2,Code-1ㆍ4ㆍ3ㆍ5 Story,17620
17620,New Z.A.2 Style Vol.1,Z.A2,Code-1ㆍ4ㆍ3ㆍ5 Story,17621
17621,일방통행,Z.A2,Code-1ㆍ4ㆍ3ㆍ5 Story,17622
17622,Listen My Love,Z.A2,Code-1ㆍ4ㆍ3ㆍ5 Story,17623
17623,Brisk Intro B,Z.A2,Code-1ㆍ4ㆍ3ㆍ5 Story,17624


In [0]:
full_data_all.to_csv("./gdrive/My Drive/Colab/data/full_data_all.csv", index=False)

## 5. 추출한 데이터 테이블 연결

In [0]:
full_data_sep = pd.read_csv("./gdrive/My Drive/Colab/data/full_data_sep.csv")
full_data_genre = pd.read_csv("./gdrive/My Drive/Colab/data/full_data_genre.csv")
full_data_era = pd.read_csv("./gdrive/My Drive/Colab/data/full_data_era.csv")
full_data_all = pd.read_csv("./gdrive/My Drive/Colab/data/full_data_all.csv")

In [20]:
len(full_data_all),len(full_data_sep), len(full_data_era), len(full_data_genre)

(17624, 17624, 17624, 17624)

In [21]:
COLUMNS_LIST

['곡명', '아티스트', '앨범', '시대', '장르', '순위/전체', '순위/시대', '순위/장르', '순위/시대/장르']

In [22]:
full_data_all.columns

Index(['곡명', '아티스트', '앨범', '순위/전체'], dtype='object')

In [23]:
full_data_sep.columns

Index(['곡명', '아티스트', '앨범', '시대', '장르', '순위/시대/장르'], dtype='object')

In [24]:
full_data_era.columns

Index(['곡명', '아티스트', '앨범', '시대', '순위/시대'], dtype='object')

In [25]:
full_data_genre.columns

Index(['곡명', '아티스트', '앨범', '장르', '순위/장르'], dtype='object')

In [26]:
# 연결 전 중복데이터 확인
full_data_sep[['곡명','아티스트','앨범']].duplicated().sum(), full_data_era[['곡명','아티스트','앨범']].duplicated().sum(), full_data_genre[['곡명','아티스트','앨범']].duplicated().sum(), full_data_all[['곡명','아티스트','앨범']].duplicated().sum()

(49, 49, 49, 49)

In [27]:
full_data_sep[full_data_sep[['곡명','아티스트','앨범']].duplicated()==True].head()

,곡명,아티스트,앨범,시대,장르,순위/시대/장르
546,세월 가면,이광조,Golden Hits Vol.2,1980,발라드,547
771,그때는 어렸나봐요,이지연,이지연 Best,1980,발라드,772
773,저 하늘에 구름따라,이광조,Golden Hits Vol.2,1980,발라드,774
942,우리네 인생,이광조,Golden Hits Vol.2,1980,발라드,943
1355,꿈에,조덕배,나의 옛날 이야기 (사랑한 女人에게),1980,록/포크,254


In [28]:
full_data_sep[full_data_sep['곡명']=='세월 가면']

,곡명,아티스트,앨범,시대,장르,순위/시대/장르
366,세월 가면,이광조,Golden Hits Vol.2,1980,발라드,367
546,세월 가면,이광조,Golden Hits Vol.2,1980,발라드,547


In [29]:
len(full_data_sep.drop_duplicates(['곡명','아티스트','앨범'])) #17624 - 49 = 17575

17575

In [0]:
full_data_sep.drop_duplicates(['곡명','아티스트','앨범'], inplace=True)

In [31]:
full_data_sep[full_data_sep['곡명']=='세월 가면']

,곡명,아티스트,앨범,시대,장르,순위/시대/장르
366,세월 가면,이광조,Golden Hits Vol.2,1980,발라드,367


In [0]:
full_data_era.drop_duplicates(['곡명','아티스트','앨범'], inplace=True)
full_data_genre.drop_duplicates(['곡명','아티스트','앨범'], inplace=True)
full_data_all.drop_duplicates(['곡명','아티스트','앨범'], inplace=True)

In [0]:
# 중복이 제거된 '곡명, 아티스트, 앨범' 을 기준으로 연결
merged_data = pd.merge(full_data_sep[['곡명','아티스트','앨범','시대','장르','순위/시대/장르']], full_data_genre[['곡명','아티스트','앨범','순위/장르']], on=['곡명','아티스트','앨범'])

In [34]:
merged_data.columns

Index(['곡명', '아티스트', '앨범', '시대', '장르', '순위/시대/장르', '순위/장르'], dtype='object')

In [0]:
merged_data = pd.merge(merged_data[['곡명','아티스트','앨범','시대','장르','순위/시대/장르','순위/장르']], full_data_era[['곡명','아티스트','앨범','순위/시대']], on=['곡명','아티스트','앨범'])

In [36]:
merged_data.columns

Index(['곡명', '아티스트', '앨범', '시대', '장르', '순위/시대/장르', '순위/장르', '순위/시대'], dtype='object')

In [0]:
merged_data = pd.merge(merged_data[['곡명','아티스트','앨범','시대','장르','순위/시대/장르','순위/장르','순위/시대']], full_data_all[['곡명','아티스트','앨범','순위/전체']], on=['곡명','아티스트','앨범'])

In [38]:
merged_data.duplicated().sum() # 중복데이터 없음

0

In [39]:
merged_data.isnull().sum() # 누락 없음

곡명          0
아티스트        0
앨범          0
시대          0
장르          0
순위/시대/장르    0
순위/장르       0
순위/시대       0
순위/전체       0
dtype: int64

In [40]:
merged_data.head()

,곡명,아티스트,앨범,시대,장르,순위/시대/장르,순위/장르,순위/시대,순위/전체
0,가로수 그늘아래 서면,이문세,5집 가로수 그늘아래서면,1980,발라드,1,157,1,291
1,사랑하기 때문에,유재하,1집 사랑하기 때문에,1980,발라드,2,161,2,305
2,너에게로 또 다시,변진섭,2집 너에게로 또다시,1980,발라드,3,288,3,555
3,가리워진 길,유재하,1집 사랑하기 때문에,1980,발라드,4,333,4,649
4,비처럼 음악처럼,김현식,3집 비처럼 음악처럼,1980,발라드,5,358,5,691


In [0]:
merged_data.to_csv("./gdrive/My Drive/Colab/data/merged_data.csv", index=False)